In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from pycaret.classification import *
from sklearn.metrics import log_loss
import random

In [3]:
train = pd.read_csv('./open/train.csv')
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 


test = pd.read_csv('./open/test.csv')
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

submit = pd.read_csv('./open/sample_submission.csv')

In [5]:
train.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NAN,2.0,-6.0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [6]:
from pycaret.classification import *
from sklearn.metrics import log_loss

clf = setup(train, target = 'credit', train_size = 0.85)

,Description,Value
0,session_id,950
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(26457, 19)"
5,Missing Values,False
6,Numeric Features,5
7,Categorical Features,13
8,Ordinal Features,False
9,High Cardinality Features,False


In [7]:
#logloss 적용 및 predict_proba로 제출하기 위해 metric 추가
add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7fc41618fca0>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [8]:
#svm, ridge는 predict_proba 미지원으로 제외
best5 = compare_models(fold = 5, sort = 'logloss', n_select = 5, exclude=['svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7047,0.7261,0.4558,0.6844,0.6459,0.2818,0.3397,0.7517,3.3260
lightgbm,Light Gradient Boosting Machine,0.7004,0.7134,0.4307,0.6928,0.6265,0.2453,0.3252,0.7591,0.2320
gbc,Gradient Boosting Classifier,0.6914,0.6563,0.4071,0.6310,0.6048,0.2050,0.2962,0.7954,1.6680
lda,Linear Discriminant Analysis,0.6442,0.6044,0.3379,0.5390,0.5114,0.0136,0.0543,0.8627,0.1440
nb,Naive Bayes,0.6432,0.6050,0.3341,0.5739,0.5042,0.0026,0.0302,0.8641,0.2760
lr,Logistic Regression,0.6427,0.5591,0.3333,0.4130,0.5028,0.0000,0.0000,0.8759,0.5220
rf,Random Forest Classifier,0.6960,0.7476,0.5372,0.6750,0.6808,0.3659,0.3711,1.0069,0.6600
ada,Ada Boost Classifier,0.6903,0.6244,0.4025,0.6311,0.5999,0.1942,0.2953,1.0808,0.1980
et,Extra Trees Classifier,0.6712,0.7071,0.5208,0.6514,0.6577,0.3191,0.3229,2.6767,0.6200
knn,K Neighbors Classifier,0.6202,0.6683,0.4692,0.6002,0.6078,0.2226,0.2246,3.8316,0.3700


In [9]:
# 블랜딩 앙상블
blended = blend_models(estimator_list = best5, fold = 5, optimize = 'logloss')
pred_holdout = predict_model(blended)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.6903,0.6976,0.4053,0.6498,0.6026,0.1972,0.2937,0.7878
1,0.6941,0.7150,0.4091,0.7134,0.6069,0.2078,0.3078,0.7801
2,0.6901,0.7078,0.4037,0.7088,0.6004,0.1945,0.2948,0.7860
3,0.6985,0.7031,0.4144,0.6540,0.6134,0.2212,0.3238,0.7823
4,0.6938,0.7051,0.4084,0.6740,0.6067,0.2111,0.3051,0.7823
Mean,0.6933,0.7057,0.4082,0.6800,0.6060,0.2064,0.3050,0.7837
SD,0.0031,0.0057,0.0037,0.0267,0.0044,0.0097,0.0109,0.0028


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6939,0.7180,0.4185,0.7175,0.6099,0.2296,0.3287,0.7811


In [10]:
#lda 와 nb는 logloss가 좋지만 accuracy, auc가 상대적으로 좋지 않아 커스텀모델 생성
custom_model = compare_models(fold = 5, sort = 'logloss', n_select = 6, include=['lightgbm','gbc','rf','ada', 'lda', 'nb'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7004,0.7134,0.4307,0.6928,0.6265,0.2453,0.3252,0.7591,0.2180
gbc,Gradient Boosting Classifier,0.6914,0.6563,0.4071,0.6310,0.6048,0.2050,0.2962,0.7954,1.7540
lda,Linear Discriminant Analysis,0.6442,0.6044,0.3379,0.5390,0.5114,0.0136,0.0543,0.8627,0.1160
nb,Naive Bayes,0.6432,0.6050,0.3341,0.5739,0.5042,0.0026,0.0302,0.8641,0.0400
rf,Random Forest Classifier,0.6960,0.7476,0.5372,0.6750,0.6808,0.3659,0.3711,1.0069,0.4740
ada,Ada Boost Classifier,0.6903,0.6244,0.4025,0.6311,0.5999,0.1942,0.2953,1.0808,0.1740


In [11]:
blended_custom = blend_models(estimator_list = custom_model, fold = 5, optimize = 'logloss')
pred_holdout_custom = predict_model(blended_custom)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.6948,0.7365,0.4114,0.6984,0.6092,0.2110,0.3097,0.7863
1,0.6970,0.7492,0.4135,0.7133,0.6117,0.2174,0.3179,0.7809
2,0.6941,0.7397,0.4105,0.7437,0.6075,0.2082,0.3088,0.7863
3,0.7011,0.7374,0.4184,0.6326,0.6180,0.2325,0.3310,0.7836
4,0.6949,0.7405,0.4102,0.6950,0.6086,0.2152,0.3087,0.7825
Mean,0.6964,0.7406,0.4128,0.6966,0.6110,0.2169,0.3152,0.7839
SD,0.0026,0.0045,0.0030,0.0363,0.0038,0.0085,0.0086,0.0021


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6972,0.7555,0.4227,0.7254,0.6146,0.2404,0.3389,0.7789


In [12]:
#자동으로 상위 5개 선택한 모델
final_model = finalize_model(blended)

#Accurary, AUC, Logloss 셋다 상위 4개인 모델 
final_model_custom = finalize_model(blended_custom)

In [13]:
#Accurary, AUC, Logloss 셋다 상위 4개인 모델 사용
prep_pipe = get_config("prep_pipe")
prep_pipe.steps.append(['trained_model', final_model_custom])
prections = prep_pipe.predict_proba(test)
prections

array([[0.12337949, 0.18707027, 0.68955024],
       [0.22763766, 0.21613827, 0.55622407],
       [0.12525134, 0.18230461, 0.69244406],
       ...,
       [0.10692997, 0.14503437, 0.74803566],
       [0.22863524, 0.20458097, 0.56678379],
       [0.1513118 , 0.28169903, 0.56698918]])

In [14]:
i, j, k = [], [], []

for row in prections:
    i.append(row[0])
    j.append(row[1])
    k.append(row[2])

len(i), len(j), len(k)

(10000, 10000, 10000)

In [15]:
submit['0'] = i
submit['1'] = j
submit['2'] = k
submit.head(10)

,index,0,1,2
0,26457,0.123379,0.187070,0.689550
1,26458,0.227638,0.216138,0.556224
2,26459,0.125251,0.182305,0.692444
3,26460,0.121451,0.155628,0.722921
4,26461,0.150794,0.245732,0.603474
5,26462,0.146502,0.242896,0.610602
6,26463,0.301053,0.428742,0.270205
7,26464,0.143793,0.234941,0.621266
8,26465,0.130551,0.206565,0.662883
9,26466,0.128404,0.255552,0.616044
